In [3]:
import pandas as pd
import numpy as np

## Считывание и сохранение больших DataFrame

### Работа с pickle

In [9]:
%%time

data = pd.read_csv('../data/blending/text_classification_train.csv')

CPU times: user 1.6 s, sys: 307 ms, total: 1.91 s
Wall time: 1.98 s


In [11]:
data.shape

(7500, 2618)

In [12]:
%%time

data.to_csv('../data/blending/text_classification_train.csv')

CPU times: user 8.63 s, sys: 449 ms, total: 9.08 s
Wall time: 9.14 s


In [13]:
%%time

pd.to_pickle(data, '../data/blending/text_classification_train.pickle')

CPU times: user 15.6 ms, sys: 27.3 ms, total: 42.9 ms
Wall time: 54.7 ms


In [14]:
%%time

data = pd.read_pickle('../data/blending/text_classification_train.pickle')

CPU times: user 17.5 ms, sys: 46.7 ms, total: 64.2 ms
Wall time: 120 ms


In [ ]:
%%time
# хорошо работает при больших данных

df.to_parquet('train.parquet', index=False)
df = pd.read_parquet('train.parquet')

Примечание: при использование to_pickle() сохраняются индексы и все типы колонок, так что при его последующим считывание датафрейм будет точно таким же

### Считывание по батчам

In [40]:
chunksize = 1000
tmp_lst = []
with pd.read_csv('../data/car_train.csv',
                 index_col='car_id',
                 dtype={'model': 'category',
                        'car_type': 'category',
                        'fuel_type': 'category',
                        'target_class': 'category'}, chunksize=chunksize) as reader:
    for chunk in reader:
        tmp_lst.append(chunk)
        
data = pd.concat(tmp_lst) 
del tmp_lst
data.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
car_id,,,,,,,,,
y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


### Используем генератор

In [ ]:
def gen(path_to_file):
#     with open(path_to_file, 'r', encoding='utf-8') as fin:
#         line = fin.readline()
    yield 1#, line

## Оптимизация памяти

### Настройка индексов

In [18]:
data = pd.read_csv('../data/blending/text_classification_train.csv')
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Columns: 2619 entries, Unnamed: 0 to labse_text_feature_767
dtypes: float64(2616), int64(1), object(2)
memory usage: 156.4 MB


In [19]:
data = pd.read_csv('../data/blending/text_classification_train.csv', index_col='Unnamed: 0')
data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Columns: 2618 entries, category to labse_text_feature_767
dtypes: float64(2616), object(2)
memory usage: 156.4 MB


### Оптимизируем числовые типы

In [20]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [21]:
data = reduce_mem_usage(data)
data.info()

Memory usage of dataframe is 149.86 MB
Memory usage after optimization is: 37.59 MB
Decreased by 74.9%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Columns: 2618 entries, category to labse_text_feature_767
dtypes: float16(2616), object(2)
memory usage: 37.6+ MB


### Оптимизируем категориальные фичи

In [91]:
df_cars = pd.read_csv('../data/car_train.csv')
df_cars.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [34]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_id         2337 non-null   object 
 1   model          2337 non-null   object 
 2   car_type       2337 non-null   object 
 3   fuel_type      2337 non-null   object 
 4   car_rating     2337 non-null   float64
 5   year_to_start  2337 non-null   int64  
 6   riders         2337 non-null   int64  
 7   year_to_work   2337 non-null   int64  
 8   target_reg     2337 non-null   float64
 9   target_class   2337 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 182.7+ KB


In [35]:
def convert_columns_to_catg(df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

In [36]:
convert_columns_to_catg(df_cars, ['model', 'car_type', 'fuel_type', 'target_class'])

converting model                          size:  0.16	->	 0.01
converting car_type                       size:  0.15	->	 0.0
converting fuel_type                      size:  0.15	->	 0.0
converting target_class                   size:  0.16	->	 0.0


In [37]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   object  
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float64 
 5   year_to_start  2337 non-null   int64   
 6   riders         2337 non-null   int64   
 7   year_to_work   2337 non-null   int64   
 8   target_reg     2337 non-null   float64 
 9   target_class   2337 non-null   category
dtypes: category(4), float64(2), int64(3), object(1)
memory usage: 120.7+ KB


In [38]:
convert_columns_to_catg(df_cars, ['car_id'])

converting car_id                         size:  0.16	->	 0.23


In [39]:
df_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   car_id         2337 non-null   category
 1   model          2337 non-null   category
 2   car_type       2337 non-null   category
 3   fuel_type      2337 non-null   category
 4   car_rating     2337 non-null   float64 
 5   year_to_start  2337 non-null   int64   
 6   riders         2337 non-null   int64   
 7   year_to_work   2337 non-null   int64   
 8   target_reg     2337 non-null   float64 
 9   target_class   2337 non-null   category
dtypes: category(5), float64(2), int64(3)
memory usage: 189.8 KB


## Ускорение при помощи Numpy

### Инициализация

In [52]:
%%timeit

a = list(range(1_000_000))

9.39 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
%%timeit

b = np.arange(1_000_000)

654 µs ± 29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [79]:
a = list(range(1_000_000))

In [80]:
b = np.arange(1_000_000)

In [81]:
%%timeit
100000 in a

475 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [82]:
%%timeit
100000 in b

360 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Поэлементные функции

In [53]:
%%timeit
[el * el for el in a]

47.4 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%%timeit
[el + 10 for el in a]

53.4 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%%timeit
b * b

670 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%%timeit
b + 10

649 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Aггрегирующие функции

In [62]:
%%timeit

max(b)

25.9 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
%%timeit

b.max()

100 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Когда использовать List?

In [67]:
%%timeit

a = np.zeros(0)
for el in range(1000):
    b = np.zeros(1000)
    a = np.append(a, b)

302 ms ± 28.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%%timeit

a = []
for el in range(1000):
    b = [0 for i in range(1000)]
    a += b

20.2 ms ± 373 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Юзаем set()

In [87]:
a = np.arange(1000)
b = np.arange(1000000) * -1

In [88]:
%%timeit
np.isin(a, b)

21.1 ms ± 530 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [89]:
st = set(b)

In [90]:
%%timeit
[el in st for el in a]

41 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Векторизация в pandas

In [96]:
df_cars.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


### Numpy.where()

In [110]:
def simple_if(x):
    if x['car_rating'] < 3.78:
        return x['car_type']
    else:
        return x['fuel_type']

In [112]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.44 ms ± 59.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
%%timeit
np.where(df_cars['car_rating'].values < 3.78, df_cars['car_type'].values, df_cars['fuel_type'].values)

19 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Numpy.vectorize()

In [114]:
def simple_if2(car_rating, car_type, fuel_type):
    if car_rating < 3.78:
        return car_type
    else:
        return fuel_type

In [117]:
vectfunc = np.vectorize(simple_if2)

In [120]:
%%timeit
vectfunc(df_cars['car_rating'], df_cars['car_type'], df_cars['fuel_type'])

349 µs ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Numpy.select()

In [138]:
import re

def hard_if(x):
    if x['car_rating'] < 3:
        if 'Audi' == x['model']:
            return 0
        else:
            return 1
    elif x['car_rating'] in [3, 4, 5]:
        return 2
    else:
        return 3

In [139]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.38 ms ± 53.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [140]:
%%timeit
conditions = [
    (df_cars['car_rating'] < 3) & (df_cars['model'] == 'Audi'),
    (df_cars['car_rating'] < 3),
    df_cars['car_rating'].isin([3, 4, 5])
]

choices = [0, 1, 2]
np.select(conditions, choices, default=3)

347 µs ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Переписываем словари

In [147]:
mydict = {'economy': 0,
          'standart': 1,
          'business': 2,
          'premium': 3}
def f(x):
    if x['car_rating'] > 5:
        return mydict[x['car_type']]
    else:
        return np.nan

In [148]:
%%timeit
df_cars.apply(simple_if, axis=1)

7.65 ms ± 75.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [151]:
%%timeit
np.where(df_cars['car_rating'] > 5, df_cars['car_type'].map(mydict), np.nan)

222 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Пишем groupby на numpy

In [172]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
df_cars['int_model'] = lbl.fit_transform((df_cars['model'] + df_cars['fuel_type']).astype(str))

In [173]:
%%timeit
df_cars.groupby(['model', 'fuel_type'])['target_reg'].sum()

340 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [174]:
%%timeit
np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])

9.63 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [175]:
%%timeit
df_cars.groupby(['model', 'fuel_type'])['target_reg'].mean()

335 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [176]:
%%timeit
sums = np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])
cnt = np.bincount(df_cars['int_model'], weights=df_cars['target_reg'])
sums / cnt

19.7 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Многопоточное apply

### Swifter

In [179]:
!pip install swifter -q

In [181]:
import swifter

In [230]:
import sys
sys.path.append('../src/')
import text_prepare
import importlib
importlib.reload(text_prepare)

<module 'text_prepare' from '/Users/sergak/Documents/Neyro_sets/Competitive_Data_Science/notebooks/../src/text_prepare.py'>

In [221]:
df = pd.read_csv('../data/blending/text_transformer_data.csv')
df.head()

,category,text
0,extreme,Ледник Пасторури это цирковой ледник расположе...
1,martial_arts,Главные участники предстоящего Betokenoid 274 ...
2,extreme,Ttokenoid Btokenoid – карта с которой можно не...
3,autosport,В Сильверстоуне произошли крупные обновления а...
4,extreme,На протяжении более чем 30 лет Вестсайд являет...


In [232]:
%%time

tmp = df['text'].apply(text_prepare.text_prepare)

CPU times: user 17.5 s, sys: 3.87 s, total: 21.3 s
Wall time: 21.6 s


In [233]:
%%time

tmp = df['text'].swifter.apply(text_prepare.text_prepare)

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

CPU times: user 19.1 s, sys: 4.24 s, total: 23.3 s
Wall time: 23.4 s


### Multiprocessing

In [220]:
from multiprocessing import  Pool

In [228]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = np.concatenate(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [231]:
%%time
tmp = parallelize_dataframe(df, text_prepare.many_row_prepare)

CPU times: user 10.2 ms, sys: 39 ms, total: 49.2 ms
Wall time: 6.41 s


### Dask

In [66]:
import dask.dataframe as dd

In [73]:
ddf = dd.read_csv('transactions_train.csv')

In [75]:
ddf.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


#### Особенность dask - .compute()

зато можно посмотреть типы данных, head, задать фильтры и только потом зачитать целиком файл, делать select по колонкам 

этот принцип называется lazy_evaluation

In [76]:
ddf = ddf[ddf.sales_channel_id==2]

In [77]:
ddf.compute()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
588584,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,902288001,0.022017,2
588585,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,856440002,0.042356,2
588586,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
588587,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2


https://docs.dask.org/en/stable/